In [ ]:
!pip uninstall -y tensorboard
!pip uninstall -y tensorflow
!pip uninstall -y autoai-libs

!pip install -U geocoder
!pip install -U geopy
!pip install -U googlemaps
!pip install -U folium
!pip install -U numpy
!pip install -U pandas

import pandas as pd
import numpy as np

from pandas import json_normalize  # tranform JSON file into a pandas dataframe

import googlemaps
from datetime import datetime
import requests
import geopy

geopy.geocoders.options.default_user_agent = "my-application"

import ast
import json
import time

import folium
from folium.plugins import HeatMap

from ibm_botocore.client import Config
import ibm_boto3

print('Ready')

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
def download_file_cos(credentials, local_file_name, key):
    cos = ibm_boto3.client(service_name='s3',
                           ibm_api_key_id=credentials['IBM_API_KEY_ID'],
                           ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
                           ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                           config=Config(signature_version='oauth'),
                           endpoint_url=credentials['ENDPOINT'])
    try:
        res = cos.download_file(Bucket=credentials['BUCKET'], Key=key, Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [ ]:
def upload_file_cos(credentials, local_file_name, key):
    cos = ibm_boto3.client(service_name='s3',
                           ibm_api_key_id=credentials['IBM_API_KEY_ID'],
                           ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
                           ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                           config=Config(signature_version='oauth'),
                           endpoint_url=credentials['ENDPOINT'])
    try:
        res = cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'], Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(' File Uploaded')

In [ ]:
download_file_cos(credentials, 'hotels_all.csv', 'hotels_all.csv')
download_file_cos(credentials, 'food_Hotels.csv', 'food_Hotels.csv')
download_file_cos(credentials, 'location_geometry.csv', 'location_geometry.csv')

In [ ]:
df_all_hotels_sorted = pd.read_csv('hotels_all.csv', sep=',', encoding='utf_8', index_col=False).reset_index()
df_all_hotels_sorted.shape

In [ ]:
df_all_hotels_sorted.head()

In [ ]:
df_location = pd.read_csv('location_geometry.csv', sep=',', encoding='utf_8', index_col=False).reset_index()[['geometry.location.lat', 'geometry.location.lng']]
df_location.head()

In [ ]:
map_center_lat = df_location['geometry.location.lat'].mean()
map_center_lng = df_location['geometry.location.lng'].mean()
print(map_center_lat)
print(map_center_lng)

In [ ]:
# create map using latitude and longitude values
map_Algarve = folium.Map(location=[map_center_lat, map_center_lng], zoom_start=10)

for ind in df_all_hotels_sorted.index:
    venue = df_all_hotels_sorted['name'][ind]

    location = df_all_hotels_sorted['location'][ind]
    try:
        location_json = ast.literal_eval(location)
        coord = pd.DataFrame(location_json['labeledLatLngs'])
        latitude = coord['lat'][0]
        longitude = coord['lng'][0]
    except KeyError:  # Some of the locations does not have 'labeledLatLngs', but just JSON string
        try:
            location_json = ast.literal_eval(location)
            latitude = location_json['lat']
            longitude = location_json['lng']
        except ValueError:  # Some of those JSON string does not have 'quotes' around the values
            latitude = location['lat']
            longitude = location['lng']

    name = u'Hotel: {}'.format(venue)
    label = folium.Popup(name, parse_html=True, max_width=2650)

    folium.CircleMarker(
        [latitude, longitude],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        # fill_color='#3186cc',
        fillOpacity=0.7).add_to(map_Algarve)

map_Algarve

In [ ]:
all_hotels2show_map = pd.DataFrame(df_all_hotels_sorted)

for ind in df_all_hotels_sorted.index:
    location = df_all_hotels_sorted['location'][ind]
    location_json = ast.literal_eval(location)
    try:
        coord = pd.DataFrame(location_json['labeledLatLngs'])
        latitude = coord['lat'][0]
        longitude = coord['lng'][0]
    except KeyError:
        latitude = location_json['lat']
        longitude = location_json['lng']

    all_hotels2show_map.loc[ind, 'latitude'] = latitude
    all_hotels2show_map.loc[ind, 'longitude'] = longitude

all_hotels2show_map_food_ratio = all_hotels2show_map[['latitude', 'longitude', 'food.ratio']]
all_hotels2show_map_nightlife_ratio = all_hotels2show_map[['latitude', 'longitude', 'nightlife.ratio']]
print(all_hotels2show_map.shape)

In [ ]:
all_hotels2show_map_food_ratio.head()

In [ ]:
all_hotels2show_map_nightlife_ratio.head()

In [ ]:
map_Algarve = folium.Map(location=[map_center_lat, map_center_lng], zoom_start=10)
HeatMap(all_hotels2show_map_food_ratio).add_to(map_Algarve)
map_Algarve

In [ ]:
map_Algarve = folium.Map(location=[map_center_lat, map_center_lng], zoom_start=10)
HeatMap(all_hotels2show_map_nightlife_ratio).add_to(map_Algarve)
map_Algarve

In [ ]:
download_file_cos(credentials,'concelhos.txt','concelhos.txt')
download_file_cos(credentials,'distritos.txt','distritos.txt')
download_file_cos(credentials,'todos_cp.txt','todos_cp.txt')
download_file_cos(credentials,'location_geometry.csv','location_geometry.csv')
download_file_cos(credentials,'hotels_all.csv','hotels_all.csv')
download_file_cos(credentials,'hotels_Resort.csv','hotels_Resort.csv')
download_file_cos(credentials,'food_Hotels.csv','food_Hotels.csv')
download_file_cos(credentials,'food_Resort.csv','food_Resort.csv')
print('Done')